In [1]:
import time
import os
import fsspec
import hydra
import matplotlib.pyplot as plt
import torch
from torch import Tensor
import zarr
from omegaconf import DictConfig, ListConfig, OmegaConf
from torch import Tensor, nn
from torch.nn.parallel import DistributedDataParallel
import torch.optim as torch_optimizers
from tqdm import tqdm
import sys
sys.path.append('.')
# from .modulus.examples.weather.unified_recipe.utils import get_filesystem
from utils import get_filesystem

from seq_zarr_datapipe import SeqZarrDatapipe
# module_path = os.path.abspath("modulus/examples/weather/unified_recipe/utils.py")
# spec = importlib.util.spec_from_file_location("utils", module_path)
# utils = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(utils)

# module_path = os.path.abspath("modulus/examples/weather/unified_recipe/seq_zarr_datapipe.py")
# spec = importlib.util.spec_from_file_location("seq_zarr_datapipe", module_path)
# seq_zarr_datapipe = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(seq_zarr_datapipe)

# Add eval to OmegaConf TODO: Remove when OmegaConf is updated
OmegaConf.register_new_resolver("eval", eval)

try:
    from apex import optimizers as apex_optimizers
except:
    raise ImportError(
        "training requires apex package for optimizer."
        + "See https://github.com/nvidia/apex for install details."
    )
from modulus import Module
from modulus.distributed import DistributedManager
from modulus.launch.logging import (
    LaunchLogger,
    PythonLogger,
    RankZeroLoggingWrapper,
)
from modulus.launch.logging.mlflow import initialize_mlflow
from modulus.launch.utils import load_checkpoint, save_checkpoint
from modulus.utils import StaticCaptureEvaluateNoGrad, StaticCaptureTraining


In [2]:
from modulus.models.graphcast.graph_cast_net import GraphCastNet
DistributedManager.initialize()
dist = DistributedManager()

model = GraphCastNet(input_dim_grid_nodes=184, output_dim_grid_nodes=83)
# model = Module.instantiate(
#         {
#             "__name__": cfg.model.name,
#             "__args__": {
#                 k: tuple(v) if isinstance(v, ListConfig) else v
#                 for k, v in cfg.model.args.items()
#             },  # TODO: maybe mobe this conversion to resolver?
#         }
#     )
model = model.to(dist.device)


/home/saptarishi.dhanuka_asp25/miniconda3/envs/modulus/lib/python3.10/site-packages/modulus/distributed/manager.py:346: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(


In [3]:
model.load('../../../../gc_weights/graphcast_0.25_13.mdlus')

/home/saptarishi.dhanuka_asp25/miniconda3/envs/modulus/lib/python3.10/site-packages/modulus/models/module.py:314: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = 

In [4]:
hydra.core.global_hydra.GlobalHydra.instance().clear()  # Clear previous hydra instances
hydra.initialize(config_path="conf")  
cfg = hydra.compose(config_name="config") 

/tmp/ipykernel_1871740/1095907270.py:2: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_path="conf")
/home/saptarishi.dhanuka_asp25/miniconda3/envs/modulus/lib/python3.10/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
# predicted_batch_norm = nn.BatchNorm2d(
#         cfg.curated_dataset.nr_predicted_variables, momentum=None, affine=False
#     ).to(dist.device)
# unpredicted_batch_norm = nn.BatchNorm2d(
#         cfg.curated_dataset.nr_unpredicted_variables, momentum=None, affine=False
#     ).to(dist.device)

In [6]:
# cfg.curated_dataset.train_dataset_filename = '/home/saptarishi.dhanuka_asp25/modulus-graphcast/modulus/examples/weather/unified_recipe/unified_recipe_datasets/graphcast_fixed_train.zarr'
# cfg.curated_dataset.val_dataset_filename = '/home/saptarishi.dhanuka_asp25/modulus-graphcast/modulus/examples/weather/unified_recipe/unified_recipe_datasets/graphcast_fixed_val.zarr'

In [7]:
fs = get_filesystem(
        cfg.filesystem.type,
        cfg.filesystem.key,
        cfg.filesystem.endpoint_url,
        cfg.filesystem.region_name,
    )
train_dataset_mapper = fs.get_mapper(cfg.curated_dataset.train_dataset_filename)
val_dataset_mapper = fs.get_mapper(cfg.curated_dataset.val_dataset_filename)

In [8]:
from graphcast_datapipes import SeqZarrDatapipe_GraphCast
val_datapipe = SeqZarrDatapipe_GraphCast(
        file_mapping=val_dataset_mapper,
        variable_groups=cfg.curated_dataset.variable_groups,
        batch_size=cfg.validation.batch_size,
        num_steps=cfg.validation.num_steps + cfg.training.nr_input_steps,
        shuffle=False,
        device=dist.device,
        process_rank=dist.rank,
        world_size=dist.world_size,
        batch=cfg.datapipe.batch,
        parallel=cfg.datapipe.parallel,
        num_threads=cfg.datapipe.num_threads,
        prefetch_queue_depth=cfg.datapipe.prefetch_queue_depth,
        py_num_workers=cfg.datapipe.py_num_workers,
        py_start_method=cfg.datapipe.py_start_method,
    )


In [9]:
cfg.curated_dataset.variable_groups

{'constants': ['land_sea_mask', 'geopotential_at_surface'], 'inputs_surface': ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'mean_sea_level_pressure', 'total_precipitation'], 'inputs_pressure_levels': ['vertical_velocity', 'geopotential', 'temperature', 'u_component_of_wind', 'v_component_of_wind', 'specific_humidity'], 'forcings': ['toa_incident_solar_radiation', 'day_progress_sin', 'day_progress_cos', 'year_progress_sin', 'year_progress_cos'], 'node_features': ['cos_latitude', 'cos_longitude', 'sin_longitude']}

In [10]:
for j, data in tqdm(enumerate(val_datapipe)):
    break

0it [00:00, ?it/s]


In [11]:
data[0]['constants'].shape, data[0]['inputs_surface'].shape, data[0]['inputs_pressure_levels'].shape, data[0]['forcings'].shape, data[0]['node_features'].shape

(torch.Size([1, 10, 2, 721, 1440]),
 torch.Size([1, 10, 5, 721, 1440]),
 torch.Size([1, 10, 6, 13, 721, 1440]),
 torch.Size([1, 10, 5, 1440, 721]),
 torch.Size([1, 10, 3, 721, 1440]))

In [12]:
constants = data[0]['constants']
inputs_surface = data[0]['inputs_surface']
inputs_pressure_levels = torch.reshape(data[0]['inputs_pressure_levels'], (1, 10, 78, 721, 1440))
forcings = data[0]['forcings'].permute((0, 1, 2, 4, 3))
node_features = data[0]['node_features']

In [13]:
input = (torch.concat((constants[0][0], forcings[0][0], inputs_surface[0][0], 
               inputs_pressure_levels[0][0], forcings[0][1], inputs_surface[0][1], 
               inputs_pressure_levels[0][1]), dim=0))

In [14]:
from graphcast_reordering import *

In [15]:
forcings = data[0]['forcings'].permute((0, 1, 2, 4, 3))

In [16]:
precip_model_input = torch.concat((input[permutation_index, :, :], node_features[0][0], forcings[0][2]), dim=0)
no_precip_model_input = torch.concat((input[permutation_index_no_precip, :, :], node_features[0][0], forcings[0][2]), dim=0)

In [17]:
def normalize_variables(variables, batch_norm):
    shape = variables.shape
    variables = variables.flatten(0, 1)
    variables = batch_norm(variables)
    variables = variables.view(shape)
    return variables

In [18]:
def unroll(model, constants, inputs_surface, inputs_pressure_levels, forcings, node_features, num_steps = 1):
    # Get number of steps to unroll
    if forcings.shape[0] < 2:
        raise ValueError("Need forcings at at least 2 different timesteps to make predictions")
    max_steps = forcings.shape[0] - 2
    model_pred_i_minus_1 = torch.concat((inputs_surface[0], inputs_pressure_levels[0]))
    model_pred_i_0 = torch.concat((inputs_surface[1], inputs_pressure_levels[1]))
    model_predicted = []
    for i in range(min(num_steps, max_steps)):
        # Create Input
        input = torch.concat((constants, forcings[i], model_pred_i_minus_1.squeeze(), forcings[i+1], model_pred_i_0.squeeze()), dim=0)
        
        # input = torch.concat((constants, forcings[i], inputs_surface[i], 
        #        inputs_pressure_levels[i], forcings[i+1], inputs_surface[i+1], 
        #        inputs_pressure_levels[i+1]), dim=0)
        
        input = torch.concat((input[permutation_index_no_precip], node_features, forcings[i+2])).unsqueeze(dim=0).to(model.device)
        
        # Store Predictions and update next steps for rollout
        model_pred_i_minus_1 = model_pred_i_0
        model_pred_i_0 = model(input)
        model_predicted.append(model_pred_i_0)

    # Stack predictions
    model_predicted = torch.stack(model_predicted, dim=1)

    return model_predicted

    

In [19]:
optimizer = torch.optim.Adam(model.parameters())

In [20]:
model.to('cpu')

GraphCastNet(
  (encoder_embedder): GraphCastEncoderEmbedder(
    (grid_node_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=184, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (mesh_node_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=3, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (mesh_edge_mlp): MeshGraphMLP(
      (model): Sequential(
        (0): Linear(in_features=4, out_features=512, bias=True)
        (1): SiLU()
        (2): Linear(in_features=512, out_features=512, bias=True)
        (3): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (grid2mesh_edge_mlp): MeshGraphMLP(
      (model): Se

In [21]:
with torch.no_grad():
    outputs = model(no_precip_model_input.unsqueeze(0).to(model.device))
    first_target = torch.concat((inputs_surface[0][2], inputs_pressure_levels[0][2]))[permutation_index_output].to(model.device)
    loss = torch.mean(torch.linalg.norm(outputs.squeeze() - first_target, ord=2.0, dim=1))
    print(loss.item())
    # loss.backward()
    # optimizer.step()

361187.4375


In [22]:
new_targets = torch.concat((inputs_surface[0][2], inputs_pressure_levels[0][2])).to(model.device)
loss = torch.linalg.norm(outputs.squeeze() - new_targets, ord=2.0, dim=1)

In [23]:
torch.mean(loss)

tensor(361185.5938)

In [24]:
with torch.no_grad():
    model_predictions = unroll(model, constants[0, 0].to(model.device), inputs_surface[0].to(model.device), inputs_pressure_levels[0].to(model.device), forcings[0].to(model.device), node_features[0, 0].to(model.device), num_steps=4)

In [25]:
def eval_forward(model, constants, inputs_surface, inputs_pressure_levels, forcings, node_features, num_steps = 1):
    # Forward pass
    with torch.no_grad:
        net_predicted_variables = unroll(model, constants[0, 0].to(model.device), inputs_surface[0].to(model.device), inputs_pressure_levels[0].to(model.device), forcings[0].to(model.device), node_features[0, 0].to(model.device), num_steps=num_steps)

        # l2 loss
        num_elements = torch.prod(torch.Tensor(list(net_predicted_variables.shape[1:])))
        label = torch.cat(inputs_surface[:num_steps], inputs_pressure_levels[:num_steps]).to('cpu')
        loss = (torch.mean(torch.pow(net_predicted_variables - label)))

    return loss, net_predicted_variables